In [1]:
from tqdm import tqdm
import re
import os
import numpy as np
import argparse
import json
import random
import jieba
import hanlp
import time
import pandas as pd
#import torch
import openpyxl
#from torch import cosine_similarity

#from text2vec import SentenceModel, cos_sim, semantic_search
from datetime import datetime
#从自写工具包中导入：修饰词判定、修饰词查找
from utils import modifier_judge, modifier_search

#device = torch.device('cuda')
#use_gpu = torch.cuda.is_available()

def read_vectors(path, topn):  # read top n word vectors, i.e. top is 10000
    lines_num, dim = 0, 0
    vectors = {}
    iw = []
    wi = {}
    #num_file = sum([1 for i in open(path, encoding='utf-8', errors='ignore')])  
    #print(num_file)
    with open(path, encoding='utf-8', errors='ignore') as f:
        first_line = True
        for line in tqdm(f):
            if first_line:
                first_line = False
                dim = int(line.rstrip().split()[1])
                continue
            lines_num += 1
            tokens = line.rstrip().split(' ')
            vectors[tokens[0]] = np.expand_dims(np.asarray([float(x) for x in tokens[1:]]),axis=0)
            #vectors[tokens[0]] = torch.tensor([float(x) for x in tokens[1:]], device=device).unsqueeze(0)
            iw.append(tokens[0])
            if topn != 0 and lines_num >= topn:
                break
    for i, w in enumerate(iw):
        wi[w] = i
    return vectors, iw, wi, dim
#Use your pre-trained file and put it in data/pre_trained_vactors
vectors_path = "../data/pre_trained_vectors/sgns.renmin.bigram"
topn = 0
vectors, iw, wi, dim = read_vectors(vectors_path, topn) 


355990it [00:16, 21161.57it/s]


In [2]:
def cos_sim(vec1, vec2):
    if vec1.all() == 0 or vec2.all() == 0:
        return np.zeros([1])[0]
    else:
        return (vec1.dot(vec2[0]) / (np.linalg.norm(vec1[0]) * np.linalg.norm(vec2[0])))[0]



def word2vec(word1):
    if word1 in vectors:
        return vectors[word1]
    else:
        #print('not find')
        first_line = True
        for character in word1:
            if first_line:
                first_line = False
                if character in vectors:
                    vector = vectors[character]
                else: 
                    vector = np.zeros([1, 300])
                continue
            if character in vectors:
                vector = np.append(vector,vectors[character], axis = 0)
        return np.mean(vector, axis = 0).reshape(1,300)

def words2vec(word_list):
    first_line = True
    for word in word_list:
        if first_line:
            first_line = False
            vector = word2vec(word)
            continue
        vector =  np.append(vector,word2vec(word), axis = 0)
    return np.mean(vector, axis = 0).reshape(1,300)


def save_file(path, the_list, num_name, t_num):
    
    with open(file =path, mode='w',encoding='utf - 8') as f_bor:
        f_bor.write(num_name + ': '+str(len(the_list))+t_num)
        for every_line_bor in the_list:
            f_bor.write(every_line_bor[0][0]+',')
            f_bor.write(every_line_bor[1][0]+',')
            for index_mode1_phrase_bor, mode1_phrase_bor in enumerate(every_line_bor[2]):
                for mode1_word_bor in mode1_phrase_bor:
                    f_bor.write(mode1_word_bor)
                if index_mode1_phrase_bor< len(every_line_bor[2])-1:
                    f_bor.write(';')
            f_bor.write(',')
            for index_mode2_phrase_bor, mode2_phrase_bor in enumerate(every_line_bor[3]):
                for mode2_word_bor in mode2_phrase_bor:
                    f_bor.write(mode2_word_bor)
                if index_mode2_phrase_bor< len(every_line_bor[3])-1:
                    f_bor.write(';') 
            f_bor.write('。')
            f_bor.write(every_line_bor[4])
            f_bor.write('\n')



1.0000000000000002
-0.5


In [3]:

def read_phrases(path):
    with open (path, 'r',  encoding='utf-8') as phrases_read:
        phrases = []
        first_line = True
        for line in phrases_read:
            if first_line:
                first_line = False
                line_number = line
                continue
            if line != '\n':
                cut = line.replace('\n', '').split('。')
                phrase = cut[0].split(',')
                sentence = cut[1]
                phrase = [tuple(x.split(';')) for x in phrase]
                phrase.append(sentence)
                phrases.append(tuple(phrase))
    return phrases, line_number




#老版：0.1*sim_v+0.4*sim_key+0.5*sim_modify
def simlirity_for_phase_mode1(phase_before, phase_now):
    #print(phase_before, phase_now)
    sim_v = cos_sim(word2vec(phase_before[0]),word2vec(phase_now[0]))
    sim_key = cos_sim(word2vec(phase_before[1]),word2vec(phase_now[1]))
    #print(sim_key, sim_v)
    sim_modify = np.zeros([1])[0]
    if len(phase_now)>2:
        if len(phase_before)>2:
            for word_now in phase_now[2:]:
                sim_modify_tem = np.zeros([1])[0]
                for word_before in phase_before[2:]:
                    tem_sim = cos_sim(word2vec(word_before),word2vec(word_now))
                    if tem_sim > sim_modify_tem:
                        sim_modify_tem = tem_sim
                sim_modify = sim_modify + sim_modify_tem
            sim_modify = sim_modify/len(phase_now[2:])
            return 0.1*sim_v+0.5*sim_key+0.4*sim_modify
        else:
            return 0.1*sim_v+0.6*sim_key
    else:
        return 0.14*sim_v+0.86*sim_key

def simlirity_for_phase_mode2(phase_before, phase_now):
    sim_key = cos_sim(word2vec(phase_before[0]),word2vec(phase_now[0]))
    sim_modify = np.zeros([1])[0]
    if len(phase_now)>1:
        if len(phase_before)>1:
            sim_modify = cos_sim(words2vec(phase_before[1:]),words2vec(phase_now[1:]))
            return 0.4*sim_key+0.6*sim_modify
        else:
            return 0.4*sim_key
    else:
        return sim_key

def simlirity_for_phrases(phrases_before, phrases_now):
    sum1 = np.zeros([1])[0]
    sum2 = np.zeros([1])[0]
    count1 = len(phrases_now[0])
    count2 = len(phrases_now[1])
    another_mode = False
    if len(phrases_now[1])> len(phrases_before[1]):
        another_mode = True
    temp_phrases_now = [list(i_now) for i_now in phrases_now]
    temp_phrases_before = [list(i_before) for i_before in phrases_before]
    #print(phrases_before, phrases_now)
    #print(temp_phrases_before, temp_phrases_now)
    if phrases_now[0][0] and phrases_now[1][0]:
        the_type = True
    else:
        the_type = False
    #现在的无mode2，过去无mode2
    #print(phrases_before, phrases_now)
    #print(phrases_before[1][0], phrases_now[1])
    while temp_phrases_now[0][0] and temp_phrases_before[0][0]:
        sim1_2 = np.zeros([1])[0]
        for index1_1, phrase_now in enumerate(temp_phrases_now[0]):            
            for index1_2, phrase_before in enumerate(temp_phrases_before[0]):
                sim_temp1 = simlirity_for_phase_mode1(phrase_before.split(' '), phrase_now.split(' '))
                #print(sim_temp1)
                if sim_temp1 > sim1_2:
                    pos1_1 = phrase_now
                    pos1_2 = phrase_before
                    sim1_2 = sim_temp1
        sum1 = sum1 + sim1_2
        if len(temp_phrases_now[0]) == 1 or len(temp_phrases_before[0]) == 1:
            break
        temp_phrases_now[0].remove(pos1_1)
        temp_phrases_before[0].remove(pos1_2)
    
    while temp_phrases_now[1][0] and temp_phrases_before[1][0]:
        sim2_2 = np.zeros([1])[0]-0.5
        pos2_1 = ''
        pos2_2 = ''
        for index2_1, phrase_now2 in enumerate(temp_phrases_now[1]):
            for index2_2, phrase_before2 in enumerate(temp_phrases_before[1]):
                sim_temp2 = simlirity_for_phase_mode2(phrase_before2.split(' '), phrase_now2.split(' '))
                if sim_temp2 > sim2_2:
                    pos2_1 = phrase_now2
                    pos2_2 = phrase_before2
                    sim2_2 = sim_temp2
        sum2 = sum2 + sim2_2
        if len(temp_phrases_now[1]) == 1 or len(temp_phrases_before[1]) == 1:
            if another_mode and pos2_1 in temp_phrases_now[1]:
                temp_phrases_now[1].remove(pos2_1)
            break
        try:
            temp_phrases_now[1].remove(pos2_1)
            temp_phrases_before[1].remove(pos2_2)
        except:
            print(phrases_before, phrases_now)
            print(temp_phrases_before, temp_phrases_now)
            print(pos2_1, pos2_2)
            print('mode2')
            print(sim_temp2)
            print(sim2_2)
            break
            
    if another_mode:
        while temp_phrases_now[1][0] and temp_phrases_before[0][0]:
            sim2_3 = np.zeros([1])[0]
            for index23_1, phrase_now23 in enumerate(temp_phrases_now[1]):
                for index23_2, phrase_before23 in enumerate(temp_phrases_before[0]):
                    sim_temp23 = simlirity_for_phase_mode2(phrase_before23.split(' ')[1:], phrase_now23.split(' '))
                    if sim_temp23 > sim2_3:
                        pos23_1 = phrase_now23
                        pos23_2 = phrase_before23
                        sim2_3 = sim_temp23
            sum2 = sum2 + sim2_3
            if len(temp_phrases_now[1]) == 1 or len(temp_phrases_before[0]) == 1:
                break
            try:
                temp_phrases_now[1].remove(pos23_1)
            except:
                print(phrases_before, phrases_now)
                print(temp_phrases_before, temp_phrases_now, pos23_1, pos23_2)
                break
    
    if count1 > 0 :
        sim1 = sum1/count1
    if count2 > 0 :
        sim2 = sum2/count2
    
    if the_type:
        return 0.77*sim1 + 0.23*sim2
    else:
        return sim1 + sim2
        
        
def similirity_for_corpus(corpus_before_borrowed, corpus_before_unborrowed, corpus_now):
    generation_candidates=[]
    borrowing_candidates=[]
    sim_dict=[]
    defussion_dict = []
    dire_dict = []
    judge_dict = []
    
    count_list = []
    kind = ''

    for line_now in corpus_now:
     
        sim_dot88 = 0
        dot88 =True
        defu_dot88 = 'null'
        dir_dot88 = 'null'           

        for line_before_borrowed in corpus_before_borrowed:
            common_elements_borrowed = list(set(line_before_borrowed[1][0].split(' ')).intersection(line_now[1][0].split(' ')))
            #print(line_now[-1]+' '+line_before[-1])
            if common_elements_borrowed:
                temp_sim_dot_borrowed = simlirity_for_phrases(line_before_borrowed[2:-1], line_now[2:-1])
             
                if dot88:
                    if temp_sim_dot_borrowed > sim_dot88:                            
                        sim_dot88 = temp_sim_dot_borrowed
                        defu_dot88 = line_before_borrowed[-1]+' '+line_now[-1]
                        dir_dot88 = line_before_borrowed[0][0]+','+line_now[0][0]
                        if sim_dot88 >= 0.88:  
                            dot88 = False
                            break
                            
        if dot88:
            for line_before_unborrowed in corpus_before_unborrowed:
                common_elements_unborrowed = list(set(line_before_unborrowed[1][0].split(' ')).intersection(line_now[1][0].split(' ')))
                #print(line_now[-1]+' '+line_before[-1])
                if common_elements_unborrowed:
                    temp_sim_dot_unborrowed = simlirity_for_phrases(line_before_unborrowed[2:-1], line_now[2:-1])
                    if temp_sim_dot_unborrowed > sim_dot88:                            
                        sim_dot88 = temp_sim_dot_unborrowed
                        defu_dot88 = line_before_unborrowed[-1]+' '+line_now[-1]
                        dir_dot88 = line_before_unborrowed[0][0]+','+line_now[0][0]
                        if sim_dot88 >= 0.88:
                            corpus_before_borrowed.insert(0, tuple(line_before_unborrowed))
                            corpus_before_unborrowed.remove(line_before_unborrowed)
                            dot88 = False
                            break
                                
        if sim_dot88 < 0.88:
            count_list.append(tuple(line_now))
            generation_candidates.append(line_now)
            kind = 'generation'
            
        if sim_dot88 >= 0.88:
            kind = 'borrowing'
            borrowing_candidates.append(line_now)
            
        sim_dict.append(sim_dot88)
        defussion_dict.append(defu_dot88)
        dire_dict.append(dir_dot88)
        judge_dict.append(kind)
                    

        

        
        
        
    return sim_dict, defussion_dict, dire_dict, judge_dict, generation_candidates, borrowing_candidates, count_list, corpus_before_borrowed, corpus_before_unborrowed


 
        
        

In [4]:
province_list =['安徽省', '山西省', '福建省', '上海市', '陕西省', '广西壮族自治区', '宁夏回族自治区', '重庆市', '浙江省', '贵州省', '湖北省', '湖南省', '青海省', '黑龙江省', '河北省', '河南省', '江西省', '天津市', '国务院', '辽宁省', '江苏省', '新疆维吾尔自治区', '山东省', '甘肃省', '四川省', '海南省', '内蒙古自治区', '北京市', '云南省', '西藏自治区', '吉林省', '广东省']
#province_list.remove('.DS_Store')
#file_name_list.remove('国务院')
print(province_list)

            
    

['安徽省', '山西省', '福建省', '上海市', '陕西省', '广西壮族自治区', '宁夏回族自治区', '重庆市', '浙江省', '贵州省', '湖北省', '湖南省', '青海省', '黑龙江省', '河北省', '河南省', '江西省', '天津市', '国务院', '辽宁省', '江苏省', '新疆维吾尔自治区', '山东省', '甘肃省', '四川省', '海南省', '内蒙古自治区', '北京市', '云南省', '西藏自治区', '吉林省', '广东省']


In [11]:
first_time = True
#calculate year by year, from 1998 to 2023
time_year = 2024
alpha ="α=0.88"

history_path = "../results/history_innovation_base/"
history_temp_path = "../results/history_temp/"

denominator_allCorpus_borrowed = []
denominator_allCorpus_unborrowed = []
if os.path.exists(history_path+'borrowed_till'+str(time_year-1)+'.txt'):
    denominator_allCorpus_borrowed, num_borrowed =read_phrases(history_path+'borrowed_till'+str(time_year-1)+'.txt')
if os.path.exists(history_path+'unborrowed_till'+str(time_year-1)+'.txt'):
    denominator_allCorpus_unborrowed, num_unborrowed =read_phrases(history_path+'unborrowed_till'+str(time_year-1)+'.txt')

temp_corpus = []
for province in province_list:
    save_path = "../results/adoptions/"+province+str(time_year)+"年政府工作报告_self_similarity.txt"
    if os.path.exists(save_path):
        adoptionCorpus, num =read_phrases(save_path)
        print(province, time_year)
        simi_list,def_list, dir_list, judge_list, generation_list, borrowing_list, count_list, denominator_allCorpus_borrowed, denominator_allCorpus_unborrowed = similirity_for_corpus(denominator_allCorpus_borrowed, denominator_allCorpus_unborrowed, adoptionCorpus)
        temp_corpus = temp_corpus + count_list
        data_dict = {"sim": simi_list, "sentences": def_list, "from":dir_list, "kinds":judge_list}
        df = pd.DataFrame(data_dict)
        excel_path = "../results/generation&borrowing_judge/"+province+str(time_year)+"年政府工作报告_generation&borrowing_judge.xlsx"
        df.to_excel(excel_path,index=False)
        print(len(denominator_allCorpus_borrowed), len(denominator_allCorpus_unborrowed))
        
        save_file("../results/generations/"+province+str(time_year)+"generations.txt", generation_list, 'generation_num', num)
        save_file("../results/borrowings/"+province+str(time_year)+"borrowings.txt", borrowing_list, 'borrowing_num', num)

denominator_allCorpus_unborrowed = denominator_allCorpus_unborrowed + temp_corpus
save_file(history_temp_path+'borrowed_till'+str(time_year)+'d.txt', denominator_allCorpus_borrowed, 'borrowed_num', '')
save_file(history_temp_path+'unborrowed_till'+str(time_year)+'d.txt', denominator_allCorpus_unborrowed, 'unborrowed_num', '')                    

                    

四川省 2024
40457 202838
海南省 2024
40488 202807
内蒙古自治区 2024
40536 202759
北京市 2024
40586 202709
云南省 2024
40621 202674
西藏自治区 2024
40639 202656
吉林省 2024
40692 202603
广东省 2024
40771 202524


In [6]:

def remove_duplicates(lst):
    return list(set(lst))

organizing_allCorpus_borrowed = []
organizing_allCorpus_unborrowed = []
de_list = []

for x in ['a', 'b', 'c', 'd']:
    organizing_allCorpus_borrowed = organizing_allCorpus_borrowed +read_phrases(history_temp_path+'borrowed_till'+str(time_year)+x+'.txt')[0]
    organizing_allCorpus_unborrowed = organizing_allCorpus_unborrowed +read_phrases(history_temp_path+'unborrowed_till'+str(time_year)+x+'.txt')[0]

print(len(organizing_allCorpus_borrowed), len(organizing_allCorpus_unborrowed))

organizing_allCorpus_borrowed = remove_duplicates(organizing_allCorpus_borrowed)
organizing_allCorpus_unborrowed = remove_duplicates(organizing_allCorpus_unborrowed)

print(len(organizing_allCorpus_borrowed), len(organizing_allCorpus_unborrowed))

for borrowed_one in organizing_allCorpus_borrowed:
    for unborrowed_one in organizing_allCorpus_unborrowed:
        if borrowed_one == unborrowed_one:
            de_list.append(borrowed_one)
            continue
            
for de_one in de_list:
    organizing_allCorpus_unborrowed.remove(de_one)

print(len(organizing_allCorpus_borrowed), len(organizing_allCorpus_unborrowed))

organizing_allCorpus_unborrowed.sort(key=lambda x: x[0][0].split(' ')[0][-4:])

save_file(history_path+'borrowed_till'+str(time_year)+'.txt', organizing_allCorpus_borrowed, 'borrowed_num', '\n')
save_file(history_path+'unborrowed_till'+str(time_year)+'.txt', organizing_allCorpus_unborrowed, 'unborrowed_num', '\n')                    
#save_file(history_path+'unborrowed_till'+str(time_year)+'unremoved.txt', organizing_allCorpus_unborrowed, 'unborrowed_num', '\n')                    


29584 184926
8580 53219
8580 51566


In [ ]:
delis, num_unborrowed =read_phrases(history_path+'unborrowed_till2023unremoved.txt')
# 在原地删除包含元素2的子列表
for sublist in delis[:20000]:
    if '2008' in sublist[0][0] :
        #print(sublist[0][0].split(' ')[0][-4:], sublist[0][0].split(' ')[1])
        delis.remove(sublist)
        
save_file(history_path+'unborrowed_till'+str(time_year)+'.txt', delis, 'unborrowed_num', '\n')                    
